In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

from utilidades.validar import validar

In [2]:
train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")

dfinicial = pd.read_csv("data/train_limpio.csv")
dftest = pd.read_csv("data/test_limpio.csv")

dfinicial2 = pd.read_csv("data/esteban/train_limpio_ciudades.csv")
dftest2 = pd.read_csv("data/esteban/test_limpio_ciudades.csv")

xg = pd.read_csv('submits/submit_xg_prueba.csv')

In [3]:
pd.set_option('display.max_columns', 23) #ver todas las columnas
pd.set_option('display.max_rows', 100) #como mucho ver 100 filas

In [14]:
def generar_subida(DFINICIAL, metodo, DFTESTEAR, nomArchivo, pasarAdisco):
    """
    X: datos de entrenamiento
    y: target de X
    metodo: algoritmo a utilizar
    """
    
    X, y = DFINICIAL.drop( columns = ['precio'] ), DFINICIAL["precio"]

    reg = metodo(normalize=True, n_jobs = -1)
    reg.fit(X,y)
    
    subida = pd.DataFrame( reg.predict(DFTESTEAR) , columns=["target"])
    subida.insert(0, "id", test["id"]) #test todavía tiene los ids
    
    subida['id'] = subida['id'].astype(np.uint32)
    subida['target'] = subida['target'].astype(np.uint32)
    
    # Heurística: si vale menos de 310 000, ponerle 310 000
    subida.loc[ subida['target'] <= 310000, 'target'] = 310000
    
    # Heurística 2: si vale más de 12 500 000, ponerle 12 500 000
    subida.loc[ subida['target'] >= 12500000, 'target'] = 12500000
    
    if pasarAdisco:
        subida.to_csv(f"submits/" + nomArchivo, index = False) 
    
    return X, y, subida

In [18]:
dfinicial1 = dfinicial[['metroscubiertos', 'antiguedad', 'precio']]
dftest1 = dftest[['metroscubiertos', 'antiguedad']]

#dfinicial1 = dfinicial2.drop( columns = [ 'anio', 'metrostotales', 'dia', 'mes', 'banos', 'centroscomercialescercanos',  'escuelascercanas'
#                           ])                                        
 
#dftest1 = dftest2.drop( columns = [  'anio', 'metrostotales', 'dia', 'mes', 'banos', 'centroscomercialescercanos', 'escuelascercanas'
#                           ])   

X, y, subida = generar_subida( dfinicial1, LinearRegression, dftest1, "prueba_esteban.csv", False)

# IMPRIME EL PROMEDIO DE DIFERENCIA CON EL MEJOR RESULTADO (EL DE XG DE JULIÁN)
# UN VALOR MENOR ES MEJOR
dftot = subida.merge(xg, on = 'id', how = 'inner')
dftot['diferencia'] = abs( dftot['target_x'] - dftot['target_y'] )
round (dftot['diferencia'].sum() / len( dftot['diferencia'] ) )

761260.0

In [6]:
#Sólo si el valor anterior es menor a 500 000, usar esto:
#validar(X, y, LinearRegression, False)